In [ ]:
!gdown  18cij6foP2ilw2NtadINSnYP6UUbenXCf

Downloading...
From: https://drive.google.com/uc?id=18cij6foP2ilw2NtadINSnYP6UUbenXCf
To: /content/scc5966.zip
100% 18.4M/18.4M [00:00<00:00, 197MB/s]


In [ ]:
!unzip scc5966.zip

Archive:  scc5966.zip
  inflating: movie_reviews.csv       
  inflating: movies_data.csv         
  inflating: test_data.csv           
  inflating: train_data .csv         
  inflating: users_data.csv          


In [ ]:
import pandas as pd 

df_movie_reviews = pd.read_csv('movie_reviews.csv')

df_movies_data = pd.read_csv('movies_data.csv')

df_test_data = pd.read_csv('test_data.csv')

df_train_data = pd.read_csv('train_data .csv')

df_users_data = pd.read_csv('users_data.csv')

In [ ]:
from tqdm import tqdm 

df_user_item = pd.DataFrame(index = df_train_data['user_id'].unique(), columns=df_train_data['movie_id'].unique())

for movie in tqdm(df_train_data['movie_id'].unique()):
  temp = df_train_data[df_train_data.movie_id == movie]
  df_user_item.loc[temp.user_id, movie] = temp.rating.to_list()

In [ ]:
df_item_review = pd.DataFrame(index = df_train_data['movie_id'].unique(), columns=['reviews'])

for movie in tqdm(df_train_data['movie_id'].unique()):
  temp = df_movie_reviews[df_movie_reviews.movie_id == movie]
  concated_reviews = ''
  for item,row in temp.iterrows():
    concated_reviews = concated_reviews + ' ' + row.text
  
  df_item_review.loc[movie]['reviews'] = concated_reviews

In [ ]:
import nltk.stem
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def has_numbers(input_string):
    return any(char.isdigit() for char in input_string)

def MyTokenizer(text):
  stopwords = nltk.corpus.stopwords.words('english')
  stemmer = nltk.stem.SnowballStemmer('english')
  wnl = WordNetLemmatizer()
  l1 = [t for t in word_tokenize(text)]
  l2 = []
  for token in l1:
    if token not in stopwords and token.isnumeric() is False and len(token) > 2 and has_numbers(token) is False:
      l2.append(token)
  l3 = [stemmer.stem(wnl.lemmatize(t)) for t in l2]
  return l3

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import time

start = time.time()

TFIDF = TfidfVectorizer(stop_words='english', ngram_range=(1, 1), min_df=1,max_features=10000, tokenizer=MyTokenizer)
sparse_tfidf = TFIDF.fit_transform(df_item_review['reviews'])

end = time.time()
time_ = end - start

print("TFIDF time: " + str(time_))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'le', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  % sorted(inconsistent)


TFIDF time: 155.371826171875


In [ ]:
df_tfidf = pd.DataFrame(sparse_tfidf.todense(), index = df_item_review.index, columns=TFIDF.get_feature_names())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cosine
import numpy as np

In [ ]:
cont = 1
dic_sim = {}

for movie1 in tqdm(df_tfidf.index):
  for movie2 in df_tfidf.index[cont:]:
    x = np.array(df_tfidf.loc[movie1].to_list())
    y = np.array(df_tfidf.loc[movie2].to_list())
    
    similarity = cosine([x],[y])[0][0]
    
    dic_sim[(movie1,movie2)] = similarity

  cont+=1

In [ ]:
import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_items_tfidf_cosine.p', 'wb') as fp:
    pickle.dump(dic_sim, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_items_tfidf_cosine.p', 'rb') as fp:
    data = pickle.load(fp)

In [ ]:
def similarity_itens(similarities, neighbor_item, item):
  
  if (item,neighbor_item) not in similarities.keys():
    similarity = similarities[(neighbor_item,item)]
  else:
    similarity = similarities[(item,neighbor_item)]
    
  return similarity
  
def k_neighbors_nearest(df, similarities, k, user, item):
  k_neighbors_similarity = [-2] * k
  k_neighbors = [-1] * k

  for neighbor_item in df.columns:
    
    ni = neighbor_item
    if item != ni and df.loc[user][ni] is not np.nan:
      
      similarity = similarity_itens(similarities, ni, item)

      for i in range(k):
        if similarity > k_neighbors_similarity[i]:
          aux = k_neighbors_similarity[i]
          k_neighbors_similarity[i] = similarity
          similarity = aux

          aux = k_neighbors[i]
          k_neighbors[i] = ni
          ni = aux

  return k_neighbors

def pred(df, similarities, k, user, item):

  k_neighbors = k_neighbors_nearest(df, similarities, k, user, item)
  
  sum = 0
  sum_similarity = 0

  for neighbor_item in k_neighbors:
    if(neighbor_item != -1): #se nao deu o numero maximo de vizinhos mais proximos
      rating_neighbor_item = df.loc[user][neighbor_item]
    
      similarity = similarity_itens(similarities, neighbor_item, item)

      sum_similarity+= similarity
      
      sum+= similarity * rating_neighbor_item
  
  if sum_similarity ==0 and sum ==0:
    return 0
  elif sum_similarity ==0 and sum !=0:
    return sum
  else:
    return sum/sum_similarity

import numpy as np

items_mean = np.mean(df_user_item)
user_mean = np.mean(df_user_item.T)
global_mean = np.nanmean(df_user_item.values.tolist())

def pred_generic(parameters):
  return pred(parameters[0], parameters[1], parameters[2], parameters[3], parameters[4])
  
def generate(parameters):
  l= []
  
  for item,row in tqdm(df_test_data.iterrows()):
    user = row['user_id']
    item = row['movie_id']
    if user not in df_train_data['user_id'].unique() and item in df_train_data['movie_id'].unique():
      l.append(items_mean[item])
    elif user in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(user_mean[user])
    elif user not in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(global_mean)
    else:
      parameters.append(user)
      parameters.append(item)

      value = pred_generic(parameters)
      if value < 1:
        value = 1
      if value > 5:
        value = 5

      l.append(value)
  
  return l

In [ ]:
k = 1
  
y_pred = generate([df_user_item, data, k])
  
df_test_data['rating'] = y_pred
df_rating = df_test_data[['id','rating']]
df_rating = df_rating.set_index('id')
name = 'FBC_TFIDF_' + str(k) + '.csv'
df_rating.to_csv('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/results-kaggle/' +name)

3970it [18:32,  3.57it/s]
